In [363]:
import gym
import numpy as np
import sys
import os
np.random.seed(44)

In [364]:
def value_iteration(env, number_of_states, number_of_actions):
    policy = np.zeros((1, number_of_states))
    value_list = np.zeros((1, number_of_states))
    old_value_list = value_list.copy()
    episode = 0
    max_change = 1
    sigma = 0.9
    while max_change > 0.01:
        episode += 1
        for s in range(number_of_states):
            assigned_value = -np.inf
            for a in range(number_of_actions):
                # get new state and its reward        
                total_cand_value = 0
                for prob, new_state, reward, done in env.P[s][a]:
                    # get new states value
                    value_new_state = old_value_list[0][new_state]
                    cand_value = 0
                    if done:
                        cand_value = reward 
                    else:
                        cand_value = reward + sigma*value_new_state
                    total_cand_value += cand_value*prob 
                        
                if total_cand_value > assigned_value:
                    assigned_value = total_cand_value
                    policy[0][s] = a
                    value_list[0][s] = assigned_value
        changes = np.abs(value_list - old_value_list)
        max_change = np.max(changes)
        old_value_list = value_list.copy()
    print("Solved in: ", episode, " episodes")
    return value_list, policy

In [403]:
def policy_iteration(env, number_of_states, number_of_actions):
    
    ## 1
    policy = np.random.randint(number_of_actions, size=(1,number_of_states))
#     policy = np.zeros((1,number_of_states))
    value_list = np.zeros((1, number_of_states))
    episode = 0
    sigma = 0.9
    
    ## 2
    policy_stable = False
    while not policy_stable:
        episode += 1
        eval_acc = True
        while eval_acc:
            eps = 0
            for s in range(number_of_states):
                # first row
                v = value_list[0][s]

                # get the new value 
                a = policy[0][s]
                total_val_new_state = 0
                for prob, new_state, reward, done in env.P[s][a]:
                    value_new_state = value_list[0][new_state]
                    # second row
                    cand_value = 0
                    if done:
                        cand_value = reward
                        # value_list[0][s] = reward
                    else:
                        cand_value = reward + sigma*value_new_state
                    total_val_new_state += cand_value*prob 
                value_list[0][s] = total_val_new_state
                    
                # third row
                eps = max(eps, np.abs(v-value_list[0][s]))
            if eps < 0.001:
                eval_acc = False


        ## 3
        policy_stable = True
        for s in range(number_of_states):

            # assign 
            old_action = policy[0][s]
            # get the argmax a here
            max_value = -np.inf
            for a in range(number_of_actions):
                # get the new value 
                total_cand_value = 0
                for prob, new_state, reward, done in env.P[s][a]:
                    value_new_state = value_list[0][new_state]
                    cand_value = 0
                    if done:
                        cand_value = reward
                    else:
                        cand_value = reward + sigma*value_new_state
                    total_cand_value += prob*cand_value
                if total_cand_value > max_value:
                    max_value = total_cand_value
                    policy[0][s] = a

            # if old-action != policy[s]
            if old_action != policy[0][s]:
                policy_stable = False
    print("Solved in: ", episode, " episodes")

    return value_list, policy       
        

In [399]:
env_name = "FrozenLake-v0"

In [404]:
env = gym.make('FrozenLake-v0')
current_state = env.reset()
value_list, policy = policy_iteration(env, env.observation_space.n, env.action_space.n)
rewards = []
env.render()

Solved in:  4  episodes

SFFF
FHFH
FFFH
HFFG


In [73]:
policy

array([[0., 3., 0., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.]])

In [48]:
policy

array([[1., 3., 2., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.]])

In [367]:
env = gym.make(env_name)
current_state = env.reset()
value_list, policy = value_iteration(env, env.observation_space.n, env.action_space.n)
rewards = []
env.render()

Solved in:  10  episodes

SFFF
FHFH
FFFH
HFFG


In [495]:

act = int(policy[0][current_state])

print(act)
new_state, reward, finished, _ = env.step(act)
rewards.append(reward)
current_state = new_state
print(finished, reward)
env.render()

0
True 0
  (Left)
SFFF
FHFH
FFFH
HFFG
